# Find catchment areas for hospitals offering mechanical thrombectomy

Wheel-and-spoke model.

In [16]:
import pandas as pd
import numpy as np

## Import data

List of stroke teams and which treatments they can provide:

In [17]:
df_stroke_teams = pd.read_csv('../data_tabular/stroke_hospitals_2022.csv')

In [18]:
df_stroke_teams.columns

Index(['Postcode', 'Hospital_name', 'Use_IVT', 'Use_MT', 'Use_MSU', 'Country',
       'Strategic Clinical Network', 'Health Board / Trust', 'Stroke Team',
       'SSNAP name', 'Admissions 21/22', 'Thrombolysis', 'ivt_rate', 'Easting',
       'Northing', 'long', 'lat', 'Neuroscience',
       '30 England Thrombectomy Example', 'hospital_city', 'Notes'],
      dtype='object')

Travel time matrix between hospitals:

In [19]:
df_time_inter_hospital = pd.read_csv('../data_tabular/inter_hospital_time_calibrated.csv', index_col='from_postcode')

In [20]:
df_time_inter_hospital.columns

Index(['B152TH', 'B714HJ', 'B95SS', 'BA13NG', 'BA214AT', 'BB23HH', 'BD96RJ',
       'BH77DW', 'BL97TD', 'BN112DH',
       ...
       'TN240LZ', 'TQ27AA', 'TR13LQ', 'TS198PE', 'TS43BW', 'WD180HB', 'WF14DG',
       'WR51DD', 'WV100QP', 'YO318HE'],
      dtype='object', length=125)

In [21]:
df_time_inter_hospital.index

Index(['B152TH', 'B714HJ', 'B95SS', 'BA13NG', 'BA214AT', 'BB23HH', 'BD96RJ',
       'BH77DW', 'BL97TD', 'BN112DH',
       ...
       'TN240LZ', 'TQ27AA', 'TR13LQ', 'TS198PE', 'TS43BW', 'WD180HB', 'WF14DG',
       'WR51DD', 'WV100QP', 'YO318HE'],
      dtype='object', name='from_postcode', length=125)

## Match MT hospitals to IVT hospitals

Pick out the names of hospitals offering MT:

In [22]:
mt_hospital_names = df_stroke_teams['Postcode'][df_stroke_teams['Use_MT'] == 1].values

mt_hospital_names

array(['RM70AG', 'E11BB', 'SW66SX', 'SE59RW', 'SW170QT', 'NW12BU',
       'NG72UH', 'CB20QQ', 'B152TH', 'CV22DX', 'ST46QG', 'L97AL',
       'PR29HT', 'M68HD', 'NE14LP', 'HU32JZ', 'LS13EX', 'S102JF',
       'BN25BE', 'BS105NB', 'PL68DH', 'OX39DU', 'SO166YD', 'CF144XW'],
      dtype=object)

Reduce columns of inter-hospital time matrix to just MT hospitals:

In [23]:
df_time_to_mt_hospital = df_time_inter_hospital[mt_hospital_names]

df_time_to_mt_hospital.columns

Index(['RM70AG', 'E11BB', 'SW66SX', 'SE59RW', 'SW170QT', 'NW12BU', 'NG72UH',
       'CB20QQ', 'B152TH', 'CV22DX', 'ST46QG', 'L97AL', 'PR29HT', 'M68HD',
       'NE14LP', 'HU32JZ', 'LS13EX', 'S102JF', 'BN25BE', 'BS105NB', 'PL68DH',
       'OX39DU', 'SO166YD', 'CF144XW'],
      dtype='object')

From this reduced dataframe, pick out the smallest time in each row and the MT hospital that it belongs to.

In [24]:
df_nearest_mt = pd.DataFrame()
# The smallest time in each row:
df_nearest_mt['time_nearest_MT'] = df_time_to_mt_hospital.min(axis='columns')
# The name of the column containing the smallest time in each row:
df_nearest_mt['name_nearest_MT'] = df_time_to_mt_hospital.idxmin(axis='columns')
# The starting hospital name:
df_nearest_mt['from_postcode'] = df_time_to_mt_hospital.index

df_nearest_mt = df_nearest_mt.set_index('from_postcode')

df_nearest_mt

,time_nearest_MT,name_nearest_MT
from_postcode,,
B152TH,0.0,B152TH
B714HJ,18.7,B152TH
B95SS,19.8,B152TH
BA13NG,33.7,BS105NB
BA214AT,74.5,BS105NB
...,...,...
WD180HB,34.8,NW12BU
WF14DG,24.1,LS13EX
WR51DD,35.9,B152TH


Save to file:

In [25]:
df_nearest_mt.to_csv('../data_tabular/nearest_mt_each_hospital.csv')